In [1]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.close('all') 

In [42]:
#########                        Inputs
    #Tickers used
#tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
    #Data timeframe
startd = datetime.datetime(2016, 1, 1) 
endd_2 = datetime.datetime(2020, 1, 1)
    #Inputs BB
bb1input = 'Close'
bb1period = 100
bb1multiplier = 0.95
bb2input = 'Close'
bb2period = 20
bb2multiplier = 0.75
bb3input = 'Close'
bb3period = 50
bb3multiplier = 0.95
bb4input = 'Close'
bb4period = 10
bb4multiplier = 0.95
    #Inputs ATR
atr1period = 1
atr1multiplier = 2
atr2period = 1
atr2multiplier = 0.75
atr3period = 10
atr3multiplier = 2
    #Inputs Decycler
d1input = 'Open'
d1period = 5
d2input = 'Open'
d2period = 50
    #Inputs SMA
sma1input = 'Close'
sma1period = 10
sma2input = 'Close'
sma2period = 5
######### 

######################           Functions
def stock_data(startd, endd, tickers=tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def ehlers_simple_decycle(df, src, hpLength, t, name):
    #hp = highpass
    upper = 0.5
    lower = 0.5
    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    alpha = 'Alpha_' + name
    hp = 'Hp_' + name
    decycler = 'Decycler_' + name
    df[(alpha, t)] = 0        
    df[(hp, t)] = 0   
    df[(decycler, t)] = 0  
    for i in range(hpLength, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, (alpha, t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, (alpha, t)] = df.loc[i-1, (alpha, t)]
        #Decycling::: src = the input!
        df.loc[i, (hp, t)] = np.square(1-(df.loc[i, (alpha, t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, (alpha, t)]) * df.loc[i-1, (hp, t)] - np.square(1-df.loc[i, (alpha, t)]) * df.loc[i-2, (hp, t)]
        df.loc[i, (decycler, t)] = src[i] - df.loc[i, (hp, t)]
    return df
    
def bolinger_bands(df, close, length, multiple):
    for i in range(len(df)):
        mean_bb = close.rolling(length).mean()
        std  = close.rolling(length).std()
        upper_bb = mean_bb + (std*multiple)
        lower_bb = mean_bb - (std*multiple)
        return mean_bb, upper_bb, lower_bb   
    
#Functions Average True Range#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, t , n, multiplier):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
#De Stoploss 2xATR implementatie:
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * multiplier)
    return Stoploss
######################

#===~~~~====~~~~~~===~~~~====~~~~~~          Start             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time1 = datetime.datetime.now().time()
print(f'1. Script Started timestamp =     {time1}')

############                       DATA
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
############

print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

######################             SELL 1
#BB1; Close, 100, 0.95
for t in tickers:
    df[('Mean_bb1', t)], df[('Upper_bb1', t)], df[('Lower_bb1', t)] = bolinger_bands(df, df[(bb1input, t)], bb1period, bb1multiplier)
    
    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] > df.loc[i, ('Upper_bb1', t)]:
            df.loc[i, ('Strat_bb1_signal', t)] = 1
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Upper_bb1', t)]:
            df.loc[i, ('Strat_bb1_signal', t)] = 0 
#ATR1; 1 periods, 2 peaks, 2 multiplier, Atrx
for t in tickers:
    df[('Stoploss_atr1', t)] = atr(df, t, atr1period, atr1multiplier)

    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        try:
            if df.loc[i, ('Stoploss1', t)] == df.loc[i-1, ('Stoploss1', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+2, ('Strat_atr1_signal', t)] = 1
            next(islice(loop, 1, 1), None)
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr1_signal', t)] = 0  
#Decycler 1; 5, Open, Green
for t in tickers:
    df = ehlers_simple_decycle(df, df[(d1input, t)], d1period, t, 'd1')
    
    for i in range(1, len(df)-1):
        if df.loc[i, ('Decycler_d1', t)] >= df.loc[i-1, ('Decycler_d1', t)]: 
            df.loc[i, ('Strat_decycle_d1_signal', t)] = 1   # Green
        else:
            df.loc[i, ('Strat_decycle_d1_signal', t)] = 0   # Red
#SMA; Close, 10~
for t in tickers:
    df[('SMA1', t)] = df[(sma1input, t)].rolling(sma1period).mean()
    
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] > df.loc[i, (sma1input, t)]:
            df.loc[i, ('Strat_sma1_signal', t)] = 1
        if df.loc[i, ('SMA1', t)] <= df.loc[i, (sma1input, t)]:
            df.loc[i, ('Strat_sma1_signal', t)] = 0
######################

############################################        SELL 2
#BB2; Close, 20, 0.75
for t in tickers:
    df[('Mean_bb2', t)], df[('Upper_bb2', t)], df[('Lower_bb2', t)] = bolinger_bands(df, df[(bb2input, t)], bb2period, bb2multiplier)

    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] > df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_bb2_signal', t)] = 1
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_bb2_signal', t)] = 0 
#ATR1; 1 periods, 1 peaks, 0.75 multiplier, Atrx
for t in tickers:
    df[('Stoploss_atr2', t)] = atr(df, t, atr2period, atr2multiplier)

    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss2', t)] = df.loc[x:i, ('Stoploss_atr2', t)].max()
        try:
            if df.loc[i, ('Stoploss2', t)] == df.loc[i-1, ('Stoploss2', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss2', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr2_signal', t)] = 1
        if df.loc[i, ('Stoploss2', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr2_signal', t)] = 0    
#Decycler 2 === SAME AS d1

#SMA; Close, 5~
for t in tickers:
    df[('SMA2', t)] = df[(sma2input, t)].rolling(sma2period).mean()
    
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA2', t)] >= df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_sma2_signal', t)] = 1
        if df.loc[i, ('SMA2', t)] < df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_sma2_signal', t)] = 0
############################################

#############################################################      SELL 3
#BB3; Close, 50, 0.95
for t in tickers:
    df[('Mean_bb3', t)], df[('Upper_bb3', t)], df[('Lower_bb3', t)] = bolinger_bands(df, df[(bb3input, t)], bb3period, bb3multiplier)

    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Mean_bb3', t)]:
            df.loc[i, ('Strat_bb3_signal', t)] = 1
        if df.loc[i, ('Close', t)] > df.loc[i, ('Mean_bb3', t)]:
            df.loc[i, ('Strat_bb3_signal', t)] = 0 
#ATR3; 10 periods, 3 peaks, 2 mutliplier
for t in tickers:
    df[('Stoploss_atr3', t)] = atr(df, t, atr3period, atr3multiplier)

    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss3', t)] = df.loc[x:i, ('Stoploss_atr3', t)].max()
        try:
            if df.loc[i, ('Stoploss3', t)] == df.loc[i-1, ('Stoploss3', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss3', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+2, ('Strat_atr3_signal', t)] = 1
            next(islice(loop, 1, 1), None)
            x=i
        if df.loc[i, ('Stoploss3', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_atr3_signal', t)] = 0   
#Decycler 3 === D1
#SMA3 === SMA 1  (dus, SMA10)
for t in tickers:  
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] < df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_sma3_signal', t)] = 1
        if df.loc[i, ('SMA1', t)] <= df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_sma3_signal', t)] = 0
#############################################################    

#####################################################################################     BUY 4
####Buy 1; False flag repurchase
#Decycler == D1
    #Buy if green
#BB == BB3
for t in tickers:
    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] >= df.loc[i, ('Upper_bb3', t)]:
            df.loc[i, ('Strat_close_bb3_signal', t)] = 1
        if df.loc[i, ('Close', t)] < df.loc[i, ('Upper_bb3', t)]:
            df.loc[i, ('Strat_close_bb3_signal', t)] = 0
####Buy 2; Bodem buy
#Decycler; 50, open
for t in tickers:
    df = ehlers_simple_decycle(df, df[(d2input, t)], d2period, t, 'd2')
    
    for i in range(1, len(df)-1):
        if df.loc[i, ('Decycler_d2', t)] >= df.loc[i-1, ('Decycler_d2', t)]: 
            df.loc[i, ('Strat_decycle_d2_signal', t)] = 1   # Green
        else:
            df.loc[i, ('Strat_decycle_d2_signal', t)] = 0   # Red    
#BB == BB3 + SMA == SMA1 (Dus sma10)
for t in tickers:
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] >= df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_sma3_buy_signal', t)] = 1
        if df.loc[i, ('SMA1', t)] < df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_sma3_buy_signal', t)] = 0
#Under BBL for 2 days
for t in tickers:
    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] < df.loc[i, ('Lower_bb3', t)] and df.loc[i-1, ('Close', t)] < df.loc[i-1, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_close_buy_signal', t)] = 1
        if df.loc[i, ('Close', t)] < df.loc[i, ('Lower_bb3', t)] and df.loc[i-1, ('Close', t)] < df.loc[i-1, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_close_buy_signal', t)] = 0
####Buy 3; tussendoor
#BB === BB3   &   BB4; Open, 10, 0.95
for t in tickers:
    df[('Mean_bb4', t)], df[('Upper_bb4', t)], df[('Lower_bb4', t)] = bolinger_bands(df, df[(bb4input, t)], bb4period, bb4multiplier)

    for i in range(1, len(df)-1):
        if df.loc[i, ('Mean_bb4', t)] > df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_bb4_signal', t)] = 1
        if df.loc[i, ('Mean_bb4', t)] <= df.loc[i, ('Lower_bb3', t)]:
            df.loc[i, ('Strat_bb4_signal', t)] = 0 
    for i in range(1, len(df)-1):
        if df.loc[i-1, ('Close', t)] <= df.loc[i-1, ('Lower_bb4', t)] and df.loc[i, ('Close', t)] > df.loc[i, ('Lower_bb4', t)]:
            df.loc[i, ('Strat_close_bb4_signal', t)] = 1
        else:
            df.loc[i, ('Strat_close_bb4_signal', t)] = 0 
##################################################################################### 
#===~~~~====~~~~~~===~~~~====~~~~~~          Half             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'3. Script Halfway timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

################################################        STRATEGIE
####Sell strategie
for t in tickers:
    df[('Strat_s', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Strat_sma1_signal', t)] == 1:
            if df.loc[i, ('Strat_decycle_d1_signal', t)] == 1:
                if df.loc[i, ('Strat_atr1_signal', t)] == 1:
                    if df.loc[i, ('Strat_bb1_signal', t)] == 1:
                        df.loc[i, ('Strat_s', t)] = 1
        if df.loc[i, ('Strat_atr2_signal', t)] == 1:
            if df.loc[i, ('Strat_sma2_signal', t)] == 1:
                if df.loc[i, ('Strat_bb2_signal', t)] == 1:
                    if df.loc[i, ('Strat_decycle_d1_signal', t)] == 1:
                        df.loc[i, ('Strat_s', t)] = 1
        if df.loc[i, ('Strat_atr3_signal', t)] == 1:
            if df.loc[i, ('Strat_sma3_signal', t)] == 1:
                if df.loc[i, ('Strat_bb3_signal', t)] == 1:
                    if df.loc[i, ('Strat_decycle_d1_signal', t)] == 0:
                        df.loc[i, ('Strat_s', t)] = 1
####Buy Strategie
for t in tickers:
    df[('Strat_b', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Strat_decycle_d1_signal', t)] == 1:
            if df.loc[i, ('Strat_close_bb3_signal', t)] == 1:
                df.loc[i, ('Strat_b', t)] = 1
        if df.loc[i, ('Strat_decycle_d2_signal', t)] == 1:
            if df.loc[i, ('Strat_sma3_buy_signal', t)] == 1:
                if df.loc[i, ('Strat_close_buy_signal', t)] == 1:
                    df.loc[i, ('Strat_b', t)] = 1
        if df.loc[i, ('Strat_bb4_signal', t)] == 1:
            if df.loc[i, ('Strat_close_bb4_signal', t)] == 1:
                df.loc[i, ('Strat_b', t)] = 1
###Totale Strategie ------------------------------------------
for t in tickers:
    i = 0  #longest sma = 20!
    while i < len(df)-1:
        if df.loc[i, ('Strat_s', t)] == 1:
            while True:
                if df.loc[i, ('Strat_b', t)] == 0:
                    df.loc[i+1, ('Strat', t)] = 0
                    i+=1
                    if i == len(df)-1:
                        break
                if df.loc[i, ('Strat_b', t)] == 1:
                    df.loc[i+1, ('Strat', t)] = 1
                    i+=1
                    break               
        if df.loc[i, ('Strat_s', t)] == 0:
            df.loc[i+1, ('Strat', t)] = 1
            i+=1
            if i == len(df)-1:
                break
################################################

#===~~~~====~~~~~~===~~~~====~~~~~~          END             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'2. Script Completed timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

1. Script Started timestamp =     14:53:31.750121
Data, amount of rows = 1005
3. Script Halfway timestamp =   14:54:10.499154
Data, amount of rows = 1005
2. Script Completed timestamp =   14:54:15.437944
Data, amount of rows = 1006


In [43]:
df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
    
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5, 0.5, 0.5], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)])
    ax[1].plot(df[('Strat', t)])
    ax[2].plot(df[('Percent_return', t)].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    ax[3].plot(df[('Strat_b', t)])
    ax[4].plot(df[('Strat_s', t)])
    r1=df[('Strat_return', t)].sum()
    r2=df[('Percent_return', t)].sum()
    print(f'Strat return {t} = {r1}')
    print(f'Hold return {t} = {r2}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.45687189758583024
Hold return SPY = 0.5041826175961925


In [23]:
print(df.index.max())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

1005


Attributes       Date   Adj Close       Close        High         Low  \
Symbols                       SPY         SPY         SPY         SPY   
0          2016-01-04  182.513626  201.020004  201.029999  198.589996   
1          2016-01-05  182.822327  201.360001  201.899994  200.050003   
2          2016-01-06  180.516174  198.820007  200.059998  197.600006   
3          2016-01-07  176.185333  194.050003  197.440002  193.589996   
4          2016-01-08  174.251419  191.919998  195.850006  191.580002   
5          2016-01-11  174.423950  192.110001  193.410004  189.820007   
6          2016-01-12  175.831238  193.660004  194.550003  191.139999   
7          2016-01-13  171.445892  188.830002  194.860001  188.380005   
8          2016-01-14  174.260468  191.929993  193.259995  187.660004   
9          2016-01-15  170.519791  187.809998  188.759995  185.520004   
10         2016-01-19  170.746765  188.059998  190.110001  186.199997   
11         2016-01-20  168.558640  185.649994  187.500000  181.020004   
12         2016-01-21  169.502914  186.690002  188.869995  184.639999   
13         2016-01-22  172.980286  190.520004  190.759995  188.880005   
14         2016-01-25  170.365448  187.639999  190.149994  187.410004   
15         2016-01-26  172.689743  190.199997  190.529999  188.020004   
16         2016-01-27  170.810318  188.130005  191.559998  187.059998   
17         2016-01-28  171.700104  189.110001  190.199997  187.160004   
18         2016-01-29  175.885727  193.720001  193.880005  189.880005   
19         2016-02-01  175.822144  193.649994  194.580002  191.839996   
20         2016-02-02  172.653442  190.160004  191.970001  189.539993   
21         2016-02-03  173.688477  191.300003  191.779999  187.100006   
22         2016-02-04  173.960876  191.600006  192.750000  189.960007   
23         2016-02-05  170.646927  187.949997  191.669998  187.199997   
24         2016-02-08  168.349808  185.419998  186.119995  182.800003   
25         2016-02-09  168.358871  185.429993  186.940002  183.199997   
26         2016-02-10  168.213638  185.270004  188.339996  185.119995   
27         2016-02-11  166.025482  182.860001  184.100006  181.089996   
28         2016-02-12  169.448425  186.630005  186.649994  183.960007   
29         2016-02-16  172.308426  189.779999  189.809998  187.630005   
30         2016-02-17  175.123047  192.880005  193.320007  191.009995   
31         2016-02-18  174.405762  192.089996  193.270004  191.720001   
32         2016-02-19  174.324066  192.000000  192.179993  190.449997   
33         2016-02-22  176.848099  194.779999  194.949997  193.789993   
34         2016-02-23  174.614578  192.320007  194.320007  192.179993   
35         2016-02-24  175.413559  193.199997  193.529999  189.320007   
36         2016-02-25  177.538147  195.539993  195.550003  192.830002   
37         2016-02-26  177.129562  195.089996  196.679993  194.899994   
38         2016-02-29  175.740417  193.559998  196.229996  193.330002   
39         2016-03-01  179.871582  198.110001  198.210007  194.449997   
40         2016-03-02  180.679596  199.000000  199.059998  197.250000   
41         2016-03-03  181.387802  199.779999  199.800003  198.110001   
42         2016-03-04  181.977966  200.429993  201.350006  199.029999   
43         2016-03-07  182.123245  200.589996  201.070007  199.250000   
44         2016-03-08  180.134842  198.399994  199.919998  198.210007   
45         2016-03-09  181.024643  199.380005  199.789993  198.429993   
46         2016-03-10  181.169907  199.539993  201.070007  197.380005   
47         2016-03-11  184.093460  202.759995  202.809998  199.520004   
48         2016-03-14  183.857391  202.500000  203.039993  201.770004   
49         2016-03-15  183.557770  202.169998  202.529999  201.050003   
50         2016-03-16  184.620056  203.339996  203.820007  201.550003   
51         2016-03-17  185.791290  204.630005  205.229996  202.770004   
52         2016-03-18  186.521408  204.380005  204.779999 

# Graphs of the signals

SELL 1, 2 en 3 graphs!

In [39]:
for t in tickers:
    fig, ax = plt.subplots(3, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Stoploss1', t)], color='purple')
    ax[0].plot(df[('Upper_bb1', t)], color='blue')
    ax[0].plot(df[('SMA1', t)], color='green')
    ax[1].plot(df[('Strat_atr1_signal', t)], color='purple')
    ax[2].plot(df[('Strat_decycle_d1_signal', t)], color='red')
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Stoploss2', t)], color='purple')
    ax[1].plot(df[('Strat_atr2_signal', t)], color='purple')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Stoploss3', t)], color='purple')
    ax[1].plot(df[('Strat_atr3_signal', t)], color='purple')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …